<a href="https://colab.research.google.com/github/NigarSultana156/499A/blob/main/objectDetectionRCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

import os
import zipfile
import numpy as np
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Paths for videos and dataset
video_paths = [
    '/content/drive/MyDrive/top-view-multi-person-tracking-2020/cam1_1hour.mp4',
    '/content/drive/MyDrive/top-view-multi-person-tracking-2020/cam2_1hour.mp4',
    '/content/drive/MyDrive/top-view-multi-person-tracking-2020/cam3_1hour.mp4',
    '/content/drive/MyDrive/top-view-multi-person-tracking-2020/cam5_1hour.mp4',
    '/content/drive/MyDrive/top-view-multi-person-tracking-2020/cam6_1hour.mp4'
]

# Extract ZIP file for labeled images
zip_file_path = '/content/drive/MyDrive/top-view-multi-person-tracking-2020/labeled_images/train.zip'
extraction_path = '/content/extracted_files'
os.makedirs(extraction_path, exist_ok=True)
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

test_zip_file_path = '/content/drive/MyDrive/top-view-multi-person-tracking-2020/labeled_images/test.zip'
test_extraction_path = '/content/extracted_files'
os.makedirs(test_extraction_path, exist_ok=True)
with zipfile.ZipFile(test_zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(test_extraction_path)

print("Files extracted to:", extraction_path, "and", test_extraction_path)

Files extracted to: /content/extracted_files and /content/extracted_files


In [7]:
# Function to load images and labels
def load_data(folder_path):
    images, labels = [], []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        image = cv2.imread(img_path)
        bbox = []  # Replace with actual bounding box coordinates
        images.append(image)
        labels.append(bbox)
    return np.array(images), np.array(labels)

# Load train and test data
train_images, train_labels = load_data(extraction_path)
test_images, test_labels = load_data(test_extraction_path)

# Split train data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42
)


In [9]:
import tensorflow_hub as hub

# Load Faster R-CNN model from TensorFlow Hub
model = hub.load("https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1")


In [10]:
def detect_objects(frame):
    # Convert the frame to a tensor
    input_tensor = tf.convert_to_tensor([frame], dtype=tf.uint8)

    # Run detection
    detections = model(input_tensor)

    # Process the detection results
    boxes = detections["detection_boxes"].numpy()[0]
    scores = detections["detection_scores"].numpy()[0]
    classes = detections["detection_classes"].numpy()[0]

    # Filter detections with a high confidence score
    detection_threshold = 0.5
    filtered_boxes = boxes[scores >= detection_threshold]

    # Draw bounding boxes on the frame
    for box in filtered_boxes:
        ymin, xmin, ymax, xmax = box
        start_point = (int(xmin * frame.shape[1]), int(ymin * frame.shape[0]))
        end_point = (int(xmax * frame.shape[1]), int(ymax * frame.shape[0]))
        frame = cv2.rectangle(frame, start_point, end_point, (255, 0, 0), 2)

    return frame


In [ ]:
captures = [cv2.VideoCapture() for _ in video_paths]
for i, capture in enumerate(captures):
    capture.open(video_paths[i])
    assert capture.isOpened()

im_width = int(captures[0].get(cv2.CAP_PROP_FRAME_WIDTH))
im_height = int(captures[0].get(cv2.CAP_PROP_FRAME_HEIGHT))
out_size = (im_width * 3, im_height * 2)

fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('/content/output_with_detection.avi', fourcc, 20, out_size)

num_frames = 1000

while num_frames > 0:
    cam1, cam2, cam3, cam4, cam5 = [capture.retrieve()[1] for capture in captures if capture.grab()]

    # Apply detection on each frame
    cam1 = detect_objects(cam1)
    cam2 = detect_objects(cam2)
    cam3 = detect_objects(cam3)
    cam4 = detect_objects(cam4)
    cam5 = detect_objects(cam5)

    shape1 = list(cam1.shape)
    shape1[1] = 160
    shape2 = list(cam1.shape)
    shape2[1] = 1280 - 160

    upper = np.hstack((np.zeros(shape1), cam4, cam5, np.zeros(shape2)))
    lower = np.hstack((cam1, cam2, cam3))

    merged_frame = np.vstack((upper, lower))
    out.write(np.uint8(merged_frame))

    if num_frames % 100 == 0:
        print("Left to merge %d frames" % num_frames)

    num_frames -= 1

out.release()


Left to merge 1000 frames
Left to merge 900 frames
Left to merge 800 frames
Left to merge 700 frames


In [ ]:
!ffmpeg -i /content/output_with_detection.avi -vcodec libx264 /content/output_with_detection.mp4

In [ ]:
from google.colab import files
files.download('/content/output_with_detection.mp4')
